# Wrangle Report

"What, if any, are the characteristics of a good boy?" 

A poignant question with no easy answers. In our effort to answer this, we'll turn to the WeRateDogs twitter account, a world renowned resource for finding the goodest boys amongst us. However, before we can find the best doggos of them all (and they are all good dogs, Brent), we need to make that data viable for use internally. In order to do that, we'll proceed through the usual three step process for wrangling our data; gathering, assessing and cleaning. Our data will come from three sources: the WeRateDogs twitter archive, contained in the csv `twitter-archive-enhanced.csv`, the image predictions we can pull down remotely from the file `image-predictions.tsv`, and external data from the Twitter API. So let's go about gathering those three sources, and then we can assess their tidiness, cleanliness, and then get them sorted so we can appreciate every pupper they contain.

### Part 1: Gathering Our Data
#### File 1: The Twitter Archive


This will actually turn out to be the easiest of the bunch, from a "gathering" standpoint. As the file was provided to us by Udacity, we can simply place it in our local directory, and import it directly into a Pandas dataframe, where it will be ready for later use:

In [503]:
# import the we rate dogs twitter archive
import pandas as pd
import numpy as np

archive = pd.read_csv('twitter-archive-enhanced.csv')
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

#### File 2: The Image Predictions


Now, we have to do a little more legwork. We can remotely 'boop' the Udacity servers using the Request library to download our second data source. From there, it's just a matter making sure the text is read properly, that Pandas recognizes the tab separated data format, and voila, we'll have our predictions.

In [504]:
import requests
import io

# pull the data down from the Udacity servers
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

# Use the encoded raw text, identify the separator to pandas, and load the dataframe,
# preds here is shorthand for 'predictions'
with open('image_predictions.tsv', mode = 'wb') as file:
    file.write(r.content)

predictions = pd.read_csv('image_predictions.tsv', sep='\t')
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


#### File 3: Additonal Twitter Details


Here's where things get a little more... ruff. We're going to have to go ahead and use the `tweepy` library to programatically access twitter for all the additional information about the tweets we were interested in from above. 


Which tweets should we grab? Well we have the ids from the Twitter archive above. While this goes back further than we need, we can pare down our results later, once we have all the possible information assembled. For now, let's grab those tweets from the `dogs` df above, write the JSON we get back from the api into a file labeled `tweet_json.txt`, and read that into a Pandas dataframe:

In [505]:
# grab the twitter data for each tweet and store it as a json array in "tweet_json.txt"
import tweepy
import json

# Remember to load your own API key information from the Twitter Developers portal.
api_key = ''
api_secret = ''
access_token = ''
access_token_secret = ''

# Set up our tweepy scraper, ensuring that we can use the right amount of request and not
# violate our rate limit:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Set up an array to catch all the tweets that have since been deleted off Twitter
deleted_tweets = []

# Grab the IDs we need
tweet_ids = df['tweet_id']
# create our file if it doesn't exist
with open('tweet_json.txt', 'w+') as f:
    for tweet_id in tweet_ids:
        # set up an exception handler in the event that we run into a tweet that no longer
        # exists, and append that tweets ID into our array above
        try:
            # print('Grabbing tweet_id {}'.format(tweet_id))
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, f)
            # write a newline to ensure that each tweet gets its own separate line in the text file:
            f.write('\n')
        # document errors when they occur:
        except tweepy.TweepError:
            # print('Tweet not found: {}'.format(tweet_id))
            deleted_tweets.append(tweet_id)
            pass

# create a blank dataframe to load our JSON data into:
cols = ['tweet_id', 'retweet_count', 'favorite_count']
api_dl = pd.DataFrame(columns=cols)
with open('tweet_json.txt') as json_file:
    for line in json_file:
        # read through the file line by line, using the keys in the JSON file
        # to pull out the appropriate data we need for our dataframe
        status = json.loads(line)
        tweet_id = status['id_str']
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        
        # append each line to our dataframe
        api_dl = api_data.append(pd.DataFrame([[tweet_id, retweet_count, favorite_count]], columns=cols))

Rate limit reached. Sleeping for: 435
Rate limit reached. Sleeping for: 656
Rate limit reached. Sleeping for: 664


In [506]:
# confirm our data is loaded as expected
api_data.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7724,36264


### Part 2: Assessing / Cleaning Our Data


While normally, you can iterate through the data wrangling process by first identifying your issues as a whole, then cleaning them, for simplicity's sake, I will be identifying the issues with my data and cleaning them as I go. These issues will come in two main forms: data quality, and data tidiness. While data quality lacks a neat and *tidy* definition, tidiness does. In particular, [as defined by Hadley Wickham](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html), the three major characteristics of tidy data are:
- Each variable forms a column
- Each observation forms a row
- Each observational unit forms a table


Working backwards, we can identify data quality issues as those that deal with the actual data itself. Missing values, improper data types, improperly enterted values all enter the realm of "quality" rather than tidy. 


Bearing these definitions in mind, I'll first lay out an outline of issues I plan to address, and then we can work down the list one by one, expounding on them in more detail and addressing the issues as they occur.


##### Data Quality Issues
- Twitter Archive Table
    1. Retweets exist in the table
    3. Extra retweet columns exist in the table
    4. Ratings have incorrect denominators
    5. IDs are not all listed as strings
    6. Dogs named 'None' and 'a'
- Image Predictions Table
    7. Prediction name columns are inconsistenly formatted.
- Twitter API Data Table
    8. Retweets and favorites should be listed as floats


##### Data Tidiness Issues
- Twitter Archive Table
    1. Dog types are listed as separate columns
- All Tables
    2. Tables can be merged to provide related information

However, firstly, before any of these can be addressed, we need to make copies of our data in case we need to revert back to the data previously loaded:

In [507]:
dogs = archive.copy()
preds = predictions.copy()
api_data = api_dl.copy()

#### Data Quality Issues: Twitter Archive Table
#### 1. Retweets exist in the table

##### Define:
Unfortunately not all the information we have concerns the pupperinos that We Rate Dogs has examined. According to the [Tweet object API](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object), the `retweeted_status_id` field will return a value in the event that the user has retweeted the post. Here we can take a look at the number of retweets:

In [508]:
# pull down the number of retweets in the dataset
tweets = dogs.shape[0]
print('Overall tweets:', dogs.shape[0])
print('Retweets:', dogs[dogs['retweeted_status_id'].notnull()].shape[0])

Overall tweets: 2356
Retweets: 181


##### Code:
Now we clean out those retweets, and verify that there are no more, and that the shape of the dataframe has now dropped all of those rose.

In [509]:
# filter out the dog retweets and check the new retweet counts
dogs = dogs[dogs['retweeted_status_id'].isnull()]

##### Test:
Taking a look back at the same metrics we used before, we can see that all retweets have been dropped:

In [510]:
print('Overall tweets:', dogs.shape[0])
print('Retweets:', dogs[dogs['retweeted_status_id'].notnull()].shape[0])

Overall tweets: 2175
Retweets: 0


Great! Sorted. Let's tackle the second issue.


#### 2. Extra Retweet Data Exists

##### Define
Now that we've gotten rid of all the actual retweets, this renders columns that used to hold information about those retweets entirely unnessecary. While this does change the strucutre of the table, it still reflect a quality rather than a tidiness issue, as it doesn't reflect an adjustment to one of the core principles of tidy data listed above.


We can take a look at the columns of the dataframe and identify which of those columns are no longer necessary:

In [511]:
dogs.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

##### Code
So it seems in particular that `retweeted_status_id`, which we just used to filter out the retweets, along with `retweeted_status_user_id`, and `retweeted_status_timestamp`, won't be necessary, as they reflect properties of the retweeted content, which we just filtered out. So we can first verify that there is no longer any data in these columns, drop them, and keep cleaning:

In [512]:
print(dogs.retweeted_status_id.value_counts())
print(dogs.retweeted_status_timestamp.value_counts())
print(dogs.retweeted_status_user_id.value_counts())

Series([], Name: retweeted_status_id, dtype: int64)
Series([], Name: retweeted_status_timestamp, dtype: int64)
Series([], Name: retweeted_status_user_id, dtype: int64)


Because all the series returned were empty, we've verified they've got no values, and can drop them accordingly:

In [513]:
dogs.drop(columns=['retweeted_status_id', 'retweeted_status_timestamp', 'retweeted_status_user_id'], axis=1, inplace=True)

##### Test
A simple look at the columns should reveal they're all gone.

In [514]:
dogs.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

#### 4. Ratings have incorrect denominators

##### Define
Taking a look at the unique values for the denominator, we can see values other than ten. We know from the documentation (as well as our genuine love for the WeRateDogs account) that the denominator is always 10, so we should correct for these mistakes.

In [515]:
dogs['rating_denominator'].value_counts()

10     2153
50        3
80        2
11        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

##### Code
Simply hard coding the denominator to what we know it should be will suffice.

In [516]:
dogs['rating_denominator'] = 10
dogs['rating_denominator'].value_counts()

10    2175
Name: rating_denominator, dtype: int64

##### Test
Running another value_counts() will show us that the only remaining value left is 10.

#### 5. IDs are listed as numbers rather than strings

##### Define
We won't actually be conducting any analysis on the values of the IDs of the tweets, and we certainly can't preform any mathematical operations on their values either. As a result, all IDs should be represented as strings, rather than integer values. We can fix this easily in the `dogs` dataframe, and whie I'm jumping ahead a bit, it's easy enough to correct for in the `preds` dataframe as well (I know it to not be an issue in the `api_data` dataframe as I assembled that one myself). Let's check out the values of these ID columns and fix 'em up!

In [517]:
print(dogs.dtypes['tweet_id'])
print(preds.dtypes['tweet_id'])

int64
int64


##### Code
Recasting the columns as strings will effectively solve the problem

In [518]:
dogs['tweet_id'] = dogs['tweet_id'].astype(str)
preds['tweet_id'] = preds['tweet_id'].astype(str)

##### Test
Checking the data types again should show the problem solved:

In [519]:
# Test our results:
print(dogs.dtypes['tweet_id'])
print(preds.dtypes['tweet_id'])

object
object


#### 6. Dogs named 'None' and 'a'


##### Define
While goodbois can have any name, some of the names are not formatted correctly in the `twitter_archive` file. While we can't extract these, we can at least get rid of the erroneous values so they're excluded from any of the name-related analysis we might conduct later on! We can do this by replacing the values we don't want with NaN, as it can be excluded from many analyses easily, and won't cause us to be confused by the 680 people who supposedly named their dog 'None'. Not what I would have gone for, but you do you I guess.

In [520]:
# Here we can identify two easy fixes, 'None' and 'a':
dogs['name'].value_counts()

None       680
a           55
Lucy        11
Charlie     11
Cooper      10
          ... 
Brudge       1
Brutus       1
Lizzie       1
Grizzie      1
Devón        1
Name: name, Length: 956, dtype: int64

##### Code
Replacing all locations where we find 'None' or 'a' with a NaN will allow these values to not be included in future analyses

In [521]:
# replace values with an acutal NaN value so we can't see them
dogs.loc[(dogs.name=="None")|(dogs.name=="a"), 'name'] = np.nan
dogs['name'].value_counts()

Lucy       11
Charlie    11
Oliver     10
Cooper     10
Tucker      9
           ..
Chelsea     1
Brudge      1
Brutus      1
Lizzie      1
Devón       1
Name: name, Length: 954, dtype: int64

#### Data Quality Issues: Image Predictions Table
#### 7. Prediction name columns don't represent their contents.


##### Define
A simple enough one to fix. Taking a look at the predictions table, if you had never seen it before, it would be hard to identify what the contents of the columns was based on their headers alone. Fixing these with more descriptive names will make analyzing the data easier. For instance 'p1', 'p2', and 'p3' don't actually offer the proper information about what their columns, the predcition confidence for the first image prediction. 

In [522]:
preds.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

##### Code
We can simply create a dictionary of the values we want to replace, pass it to the .rename function, and be on our merry way.

In [523]:
preds.rename(columns={'p1': 'image_prediction1',
                      'p2': 'image_prediction2',
                      'p3': 'image_prediction3',
                      'p1_conf': 'prediction1_confidence',
                      'p2_conf': 'prediction2_confidence',
                      'p3_conf': 'prediction3_confidence',
                      'p1_dog': 'is_prediction1_dog',
                      'p2_dog': 'is_prediction2_dog',
                      'p3_dog': 'is_prediction3_dog'}, inplace=True)

##### Test
With the mapping complete, our columns variable should now reflect the updated values

In [524]:
preds.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'image_prediction1',
       'prediction1_confidence', 'is_prediction1_dog', 'image_prediction2',
       'prediction2_confidence', 'is_prediction2_dog', 'image_prediction3',
       'prediction3_confidence', 'is_prediction3_dog'],
      dtype='object')

#### Data Quality Issues: Twitter Data Table
#### 8. Retweets and Favorites Represented as Strings

##### Define
While I could have fixed this in the process of wrangling my data, it is still an issue with the data set as it currently exists. As of right now, all the data in this `api_data` dataframe is stored as a string. As we addressed above, this is fine for the `tweet_id` field, as we won't be preforming any numerical analysis on the ID values. However, as we will be doing that for both the `retweet_count` and `favorite_count` fields, we'll need to make an adjustment. In this case, it makes the most sense to use integer, as we're interested only in whole number values for each of these variables. While it might enable more precision for averages, you can't ultimately wind up with half a retweet or a favorite, only whole number amounts.

In [525]:
# verify that the columns are strings (object representation)
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 0
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2 non-null      object
 1   retweet_count   2 non-null      int64 
 2   favorite_count  2 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 64.0+ bytes


##### Code
Recasting the columns as we did above for strings should suffice

In [526]:
# convert the columns to integers
api_data[['retweet_count', 'favorite_count']] = api_data[['retweet_count', 'favorite_count']].astype(int)

##### Test
Rechecking the data types will show us the conversion was successful:

In [527]:
api_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 0
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2 non-null      object
 1   retweet_count   2 non-null      int64 
 2   favorite_count  2 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 64.0+ bytes


With several of the glaring data quality issues out of the way, now we can address the overall structure of our files, and in turn, their respective tidiness (or lack thereof).

#### Data Tidiness Issues
#### 1. Dog types represented across columns
##### Define
Back in the `dogs` dataframe (the `twitter-archive` dataframe), if we take a look, we can see that information about the dog types included spans across multiple columns.

In [528]:
dogs[['doggo', 'floofer', 'pupper', 'puppo']].head()

,doggo,floofer,pupper,puppo
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None


Instead, we can handle this data by merging these pieces of information into a new column we can call `dog_stage` which represents the values of all of these columns combined. Doing this is a little more involved, but will allow this particular dataframe to comply by a standard of tidy data, namely that "each variable forms a column". So let's do that!


##### Code
First, we'll take a look and see if we need to do any additional transformation to the values in the columns themselves:

In [529]:
print(dogs.doggo.value_counts())
print(dogs.floofer.value_counts())
print(dogs.pupper.value_counts())
print(dogs.puppo.value_counts())

None     2088
doggo      87
Name: doggo, dtype: int64
None       2165
floofer      10
Name: floofer, dtype: int64
None      1941
pupper     234
Name: pupper, dtype: int64
None     2150
puppo      25
Name: puppo, dtype: int64


Here it looks as though there's only a "None" value, or the string that represents the column itself (e.g. 'doggo', 'floofer', etc). It seems like the thing to do here is to stitch these columns together, and replace the instances we find where multiple matches occur in the same rows. In order to do that, we can replace the 'None' values with an empty string, concatenate all the columns together, and adjust for the overlapping examples we find.

In [530]:
# Replace None values with empty strings to concatenate
for col in ['doggo', 'floofer', 'pupper', 'puppo']:
    dogs.loc[dogs[col]=="None", col] = ''

# construct a column which concatenates all four columns together
dogs['dog_stages'] = dogs['doggo'] + dogs['floofer'] + dogs['pupper'] + dogs['puppo']

In [531]:
# Review overlapping columns to correct with a dictionary
dogs.dog_stages.value_counts()

                1831
pupper           224
doggo             75
puppo             24
doggopupper       10
floofer            9
doggopuppo         1
doggofloofer       1
Name: dog_stages, dtype: int64

In [532]:
# Replace values with dictionary
dogs.dog_stages.replace({'doggopupper': 'doggo, pupper',
                         'doggopuppo': 'doggo, puppo',
                         'doggofloofer': 'doggo, floofer'}, inplace=True)

#### Test

Looking at the value counts should confirm those values have been replace, the columns have been merged properly, we can drop the no longer needed columns and can move on.

In [533]:
# verify that all the values have been fixed
dogs.dog_stages.value_counts()

                  1831
pupper             224
doggo               75
puppo               24
doggo, pupper       10
floofer              9
doggo, floofer       1
doggo, puppo         1
Name: dog_stages, dtype: int64

Now that all the columns have been merged properly, we can drop the no longer needed columns and move on to the next issue!

In [534]:
# drop columns
dogs.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

#### 2. Tables can be merged to provide related information
##### Define
Two of the rules of tidy data observed above are:
- Each observation forms a row
- Each type of observational unit forms a table


In this case, we've got three tables each containing data regarding individual tweets. While they all contain separate information about those tweets, the observational units are all the same, and as such, according to the rules of tidy data they should be merged for the best analysis. So, we can link all three dataframes together using the `tweet_id` field, which all three data frames have, and export our data, clean and ready for new analyses!


However, one thing we do have to be aware of is that we don't have perfectly overlapping datasets, so in this case, if we want to ensure our data stays clean, we'll have to conduct an 'inner merge', rather than a left or right join, effectively making our final compiled dataset the intersection of the three datasets, where each `tweet_id` is present in all three.


Then we can write our file to a master .csv file which we can then use to preform any analysis on going forward.

##### Code

In [535]:
# merge the three datasets together using the formatted tweet_id fields from before
twitter_archive_master = dogs.merge(preds, on='tweet_id').merge(api_data, on='tweet_id')

#### Test
Looking at the information for the new dataframe we should see the columns from all the dataframes above. I'll also use this cell to write the dataframe to the output it needs to take:

In [536]:
twitter_archive_master.to_csv('twitter_master_archive.csv', index=False)
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   tweet_id                2 non-null      object 
 1   in_reply_to_status_id   0 non-null      float64
 2   in_reply_to_user_id     0 non-null      float64
 3   timestamp               2 non-null      object 
 4   source                  2 non-null      object 
 5   text                    2 non-null      object 
 6   expanded_urls           2 non-null      object 
 7   rating_numerator        2 non-null      int64  
 8   rating_denominator      2 non-null      int64  
 9   name                    2 non-null      object 
 10  dog_stages              2 non-null      object 
 11  jpg_url                 2 non-null      object 
 12  img_num                 2 non-null      int64  
 13  image_prediction1       2 non-null      object 
 14  prediction1_confidence  2 non-null      float6

### Conclusion
While every single issue with the dataset hasn't been addressed here, it certainly leaves us with a clean enough data file that we can start to do some real work on, and start to identify what separates the good bois from the best bois! Happy analyzing!